In [9]:
import geopandas as gpd
import requests
import uuid

In [22]:
all_passes = requests.get('https://raw.githubusercontent.com/ankoure/EastCoastSkiing/refs/heads/main/AllResorts.geojson')
all_passes_gdf = gpd.read_file(all_passes.text,driver='GeoJSON')

c:\Users\andko\anaconda3\envs\ski\Lib\site-packages\geopandas\io\file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")
c:\Users\andko\anaconda3\envs\ski\Lib\site-packages\geopandas\io\file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


In [26]:
all_passes_gdf['id'] = all_passes_gdf['trailmapurl'].str.replace('https://skimap.org/skiareas/view/','')

In [28]:
all_passes_gdf = all_passes_gdf[['id','partnered','passaffiliation']]

In [30]:
all_passes_gdf = all_passes_gdf[all_passes_gdf['passaffiliation'].isin(['Indy','Ikon','Epic'])]

In [31]:
all_passes_gdf

,id,partnered,passaffiliation
25,522,False,Ikon
30,100,True,Epic
31,95,False,Indy
32,2177,False,Indy
33,39,True,Epic
...,...,...,...
615,539,False,Ikon
616,448,True,Ikon
617,229,False,Epic
620,538,False,Ikon


In [ ]:
r = requests.get('https://skimap.org/skiareas/index.geojson')


In [13]:
gdf = gpd.read_file(r.text, driver='GeoJSON')

In [14]:
gdf

,scalerank,id,name,status,activities,official_website,geometry
0,8,1,Smokey Mountain Ski Club,operating,[downhill],http://www.skismokey.ca/,POINT (-66.92094 52.97795)
1,8,2,Northern Lights Ski Club,operating,[downhill],https://www.facebook.com/northern.lights.ski.c...,POINT (-64.02009 53.52358)
2,8,3,White Hills,operating,[downhill],http://www.whitehillsresort.com/,POINT (-54.04647 48.16178)
3,8,5,Marble Mountain,operating,[downhill],http://www.skimarble.com/,POINT (-57.82722 48.93667)
4,8,6,Snow Valley Ski Club,operating,[downhill],http://www.snowvalley.ca,POINT (-113.55668 53.48420)
...,...,...,...,...,...,...,...
5139,8,17944,Cricket Hill Trails,operating,[nordic],https://www.facebook.com/LNSUCricketHillTrails,POINT (-72.63491 44.60926)
5140,8,17946,Skilift Hemberg,operating,[downhill],https://www.skilifthemberg.ch/,POINT (9.17302 47.28233)
5141,8,17947,Skilift Degersheim,operating,[downhill],https://www.skilift-degersheim.ch/,POINT (9.19722 47.36452)
5142,8,17948,Skilift Hochwacht Mosnang,operating,[downhill],https://www.skiliftmosnang.ch/,POINT (9.04252 47.35873)


In [15]:
#Break out Nordic/Downhill to True False Columns
gdf['alpine'] = gdf['activities'].apply(lambda x: True if 'downhill' in x else False)
gdf['nordic'] = gdf['activities'].apply(lambda x: True if 'nordic' in x else False)
#Drop the original activities column
gdf.drop(columns=['activities'], inplace=True)
#gdf['nordic'].value_counts()


In [16]:
gdf = gdf.rename(columns={'status':'operatingstatus','official_website':'website'})

In [18]:
gdf['skiarea_pk'] = [uuid.uuid4() for _ in range(len(gdf.index))]

In [36]:
gdf['id']=gdf['id'].astype(int)
all_passes_gdf['id']=gdf['id'].astype(int)

gdf = gdf.merge(all_passes_gdf, on = ['id'], how = 'left')


C:\Users\andko\AppData\Local\Temp\ipykernel_18712\500824586.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_passes_gdf['id']=gdf['id'].astype(int)


In [37]:
gdf = gdf[['skiarea_pk','id','scalerank','passaffiliation','partnered','name','operatingstatus','alpine','nordic','website','geometry']]

In [ ]:
gdf = gdf.where(gdf.notna(), None)


In [45]:
gdf.rename_geometry('location',inplace=True)

In [48]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
load_dotenv()

#Write the data to PostGIS
db_connection_url = os.getenv('DATABASE_URL')

# Create a SQLAlchemy engine
engine = create_engine(db_connection_url)
# Write the GeoDataFrame to PostGIS
gdf.to_postgis(name='ski_resorts_skiarea', con=engine, if_exists='append')



In [42]:
gdf.passaffiliation.unique()

array([None, 'Ikon', 'Epic', 'Indy'], dtype=object)

In [44]:
gdf.partnered.unique()

array([None, False, True], dtype=object)

In [46]:
gdf.head()

,skiarea_pk,id,scalerank,passaffiliation,partnered,name,operatingstatus,alpine,nordic,website,location
0,860c3cc5-854d-4784-8b9d-ac3cfbef5af7,1,8,None,None,Smokey Mountain Ski Club,operating,True,False,http://www.skismokey.ca/,POINT (-66.92094 52.97795)
1,7bc94484-ba1f-42e5-b1db-f0225119b1fb,2,8,None,None,Northern Lights Ski Club,operating,True,False,https://www.facebook.com/northern.lights.ski.c...,POINT (-64.02009 53.52358)
2,9fda64b2-ea29-4c93-be1f-e9238220a6de,3,8,None,None,White Hills,operating,True,False,http://www.whitehillsresort.com/,POINT (-54.04647 48.16178)
3,9429dbbb-6a74-4a29-82fd-74ff5d01629d,5,8,None,None,Marble Mountain,operating,True,False,http://www.skimarble.com/,POINT (-57.82722 48.93667)
4,42db0976-77c3-4f17-b038-7b8c5857c699,6,8,None,None,Snow Valley Ski Club,operating,True,False,http://www.snowvalley.ca,POINT (-113.55668 53.48420)
